# Klasyfikacja Gaussowskim i wielomianowym klasyfikatorami Bayesowskimi i SVC opartymi o reprezentacje zliczające tokeny (Baf of Words i Bag of n-grams)

Ten plik umożliwa testowanie klasyfikatorów na pojednyczym zbiorze danych, tworzenie własnych zbiorów i generowanie wyników wyjaśnialności.

###Działania przygotowawcze: podłączenie Dysku Google, instalacja wymaganych pakietów, dołączenie katalogu projektu do ścieżki systemowej i import potrzebnych klas z projektu



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/authorship_anaysis/authorship_analysis/training_models

Mounted at /content/gdrive
/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis/training_models


In [ ]:
! pip install -r ../requrements.txt
! pip install https://github.com/kpu/kenlm/archive/master.zip
! python -m spacy download pl_core_news_lg

In [ ]:

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
print(sys.path)

/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis
['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/gdrive/MyDrive/authorship_anaysis/authorship_analysis']


In [ ]:
from data_preparation import CorpusPreparation
from pipelines import Pipeline, Explain
from data_fetchers import EpochsFetcher, BookSet

### Określenie listy autorów i wczytanie listy tytułów tekstów

Komentarze:
- Wśród autorów pojawia się Daniel Naborowski, jednak jego zbór tekstów jest tak ubogi, że zostaje pominięty (nie jest uwzgędniony w tekście pracy).
- Ten fragment kodu wymaga podłączenia do internetu, bo lista ksiażek jest ściągana z Wolnych Lektur.

In [ ]:
bookset = BookSet()
bookset.fetch()
kinds = ['Liryka']
if kinds[0]=='Epika':
  authors = ['Eliza Orzeszkowa', 'Henryk Sienkiewicz', 'Bolesław Prus']
  books_list = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors, kinds))
  books_epoch_list = [books_list]
if kinds[0]=='Liryka':
  authors = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
  authors_b = ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka']
  authors_r = ['Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid']
  authors_p = ['Adam Asnyk', 'Maria Konopnicka']
  authors_m = ['Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
  books_list_b = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_b, kinds))
  books_list_r = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_r, kinds))
  books_list_p = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_p, kinds))
  books_list_m = bookset.list_of_books_to_author_title_list(bookset.get_books_by_authors_list_kinds(authors_m, kinds))
  books_epoch_list = [books_list_b, books_list_r, books_list_p, books_list_m]
print(books_epoch_list)

[[('Mikołaj Sęp Szarzyński', 'Do Anusie (Anusiu! byś mie tem chciała darować...)'), ('Mikołaj Sęp Szarzyński', 'Do Anusie (Moja nadobna dzieweczko, moje kochanie!)'), ('Mikołaj Sęp Szarzyński', 'Do Anusie (Siebie muszę, nie ciebie, w tej mierze winować...)'), ('Mikołaj Sęp Szarzyński', 'Do Kasie (Im pilniej na twoje oblicze nadobne...)'), ('Mikołaj Sęp Szarzyński', 'Do Kasie (Jako lód taje przezroczysty z lekka...)'), ('Mikołaj Sęp Szarzyński', 'Do Kasie (Jeśli władną co nami błądzących gwiazd siły...)'), ('Mikołaj Sęp Szarzyński', 'Do Kasie (Mam nadzieję, że się nade mną zlitujesz...)'), ('Mikołaj Sęp Szarzyński', 'Do Zosie (Będę się zawsze dziwował twojej pikności...)'), ('Mikołaj Sęp Szarzyński', 'Do Zosie (Nie psuj niepotrzebnemi łzami wdzięcznych oczy!)'), ('Elżbieta Drużbacka', 'Awersya pewnej Damy całe życie bawiącej u wielkich Dworów, przy których gdy nie tylko same próżność, nieszczerość, zazdrość i wiele defektów uznała, waletę życiu takiemu wypowiada, a na spokojniejsze się 

### Zbiory treningowe i testowe

Lista dostępnych zbiorów zapisanych jako pickle. Zbiory oznaczone są liczbami naturalnymi i cyfra dziesiątek oznacza sposób podziału zbioru na zbiory testowe i uczące z określoną liczbą słów w próbce, a cyfra jednosci oznacza wybrany preprocessing.

In [ ]:
from os.path import exists
import pickle
datasets_filepath = 'datasets/features_'
for i in range(100):
  path = datasets_filepath + str(i)
  if exists(path):
    print(path)
    with open(path, 'rb') as f:
      data = pickle.load(f)
      head_data, _, _, _, _ = data
      print(f'test_size: {head_data[0]}, train_size: {head_data[1]}, number of words in paragraph: {head_data[2]}, preprocessing operations: {head_data[3]}, authors: {head_data[4]}')

datasets/features_0
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/features_1
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize', 'lower_text', 'remove_stop_words'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']
datasets/features_2
test_size: 20, train_size: 100, number of words in paragraph: 15, preprocessing operations: ['anonymize'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz'

In [ ]:
read_from_frozen_dataset = False # wybór zamrożonych datasetów z określonym preprocessingiem, ale podział definiuje się samodzielnie
use_prepared_dataset = True  # wybór skorzystania ze zbioru z określonym podziałem i preprocessingiem
number_of_dataset = 15 # numer wybranego zbioru zapisanego jako pickle
save_dataset = False # zbiór można zapisać jako kolejny pickle
save_number = 55 #numer najlepiej wybrac spoza wykorzystanych numerów, bo się nadpisze

mixed = True # określenie, czy książki ze zbioru treningowego mogą być też w zbiorze testowym
test_size = 200 #liczba próbek
train_size = 1000 #liczba próbek
words_num_in_par = 300 #liczba słów w próbce
representations = ['bow']  # 'bow', 'bo2', 'bo3'
models = [ 'svc_'] # 'cnb, 'svc', gnb', 'svc_'
preprocessing_list = ['anonymize', 'lower_text','POS_leave_only', 'lemmatize_text', 'remove_punctuation'] #'lower_text' / 'POS_leave_only' / 'lemmatize_text' / 'remove_punctuation' /'remove_stop_words' / 'anonymize'
prep_descr = 'alomp' # 'a' / 'alo' / 'alp' / 'als' / 'alomp_NVAdj' # skrót do nazw pliku
kinds_descr = 'l' # 'e' / 'l' # skrót do nazw pliku

In [ ]:
if use_prepared_dataset:
  file_name_read = datasets_filepath + str(number_of_dataset)
  with open(file_name_read, 'rb') as f:
    data = pickle.load(f)
    head_data, train_set, train_classes, test_set, test_classes = data
    test_size, train_size, words_num_in_par, preprocessing_list, authors = head_data
elif read_from_frozen_dataset:
  cp = CorpusPreparation()
  cp.read_corpus_form_frozen_dataset(authors, prep_descr, kinds_descr)
  cp.split_books(words_num_in_par)
  if mixed:
      train_set, train_classes, test_set, test_classes = cp.prepare_corpus_mixed_books(train_size, test_size)
  else:
      train_set, train_classes, test_set, test_classes = cp.prepare_corpus_divide_books(train_size, test_size)
else: # datasety tworzone od początku
  cp = CorpusPreparation(books_epoch_list)
  cp.get_books()
  cp.books_preprocessing(preprocessing_list)
  cp.split_books(words_num_in_par)
  if mixed:
      train_set, train_classes, test_set, test_classes = cp.prepare_corpus_mixed_books(train_size, test_size)
  else:
      train_set, train_classes, test_set, test_classes = cp.prepare_corpus_divide_books(train_size, test_size)
print(f'test_size: {test_size}, train_size: {train_size}, number of words in paragraph: {words_num_in_par}, preprocessing operations: {preprocessing_list}, authors: {authors}')

test_size: 10, train_size: 48, number of words in paragraph: 30, preprocessing operations: ['anonymize', 'lower_text', 'POS_leave_only', 'lemmatize_text', 'remove_punctuation'], authors: ['Daniel Naborowski', 'Mikołaj Sęp Szarzyński', 'Elżbieta Drużbacka', 'Adam Mickiewicz', 'Juliusz Słowacki', 'Cyprian Kamil Norwid', 'Adam Asnyk', 'Maria Konopnicka', 'Kazimierz Przerwa-Tetmajer', 'Bolesław Leśmian', 'Jan Kasprowicz']


In [ ]:
if save_dataset:
  file_name_save = datasets_filepath + str(save_number)
  with open(file_name_save, 'wb') as f:
    head_data = [test_size, train_size, words_num_in_par, preprocessing_list, authors]
    data = [head_data, train_set, train_classes, test_set, test_classes]
    pickle.dump(data, f)

Lista autorów istniejących w zbiorach treningowym i testowym

In [ ]:
existing_authors_set = set(train_classes)
authors_max_books = {}
authors = []
for books_epoch in books_epoch_list:
    authors_list = [author for (author, title) in books_epoch]
    authors_set = sorted(list(set(authors_list)))
    for author in authors_set:
        if author in existing_authors_set:
            books_num = authors_list.count(author)
            if author in authors:
                if books_num > authors_max_books[author]:
                    authors.remove(author)
                    authors_max_books[author] = books_num
                    authors.append(author)
            else:
                authors.append(author)
                authors_max_books[author] = books_num
authors


['Elżbieta Drużbacka',
 'Mikołaj Sęp Szarzyński',
 'Adam Mickiewicz',
 'Cyprian Kamil Norwid',
 'Juliusz Słowacki',
 'Adam Asnyk',
 'Maria Konopnicka',
 'Bolesław Leśmian',
 'Jan Kasprowicz',
 'Kazimierz Przerwa-Tetmajer']

### Testy klasyfikatorów

In [ ]:
p = Pipeline(
    corpus_train=train_set,
    corpus_test=test_set,
    classes_train=train_classes,
    classes_test=test_classes,
    class_names=authors,
    representations=representations,
    models=models)
p.pipelines()

In [ ]:
p.print_accuracy()

In [ ]:
p.print_confussion_matrix()

In [ ]:
d = ''
if not mixed:
    d = '_div'
path = f'../figures/train_fig{train_size}_{test_size}_{words_num_in_par}{d}_{prep_descr}_{kinds_descr}.jpg'
p.save_img(path)

In [ ]:
def conf_path():
  d = ''
  if not mixed:
      d = '_div'
  path = f'../figures/aut_features_fig{train_size}_{test_size}_{words_num_in_par}{d}_{prep_descr}_{kinds_descr}.jpg'
  return path

def lime_path(true_class: str):
  return f'../figures/explanation/lime/train_lime_fig{train_size}_{test_size}_{words_num_in_par}{d}_{prep_descr}_{kinds_descr}_{true_class}.jpg'


###Wyjaśnmialność -- LIME

In [ ]:
expl = Explain(p)

In [ ]:
true_author = 'Henryk Sienkiewicz'
#true_author = 'Adam Mickiewicz'
#true_author = 'Juliusz Słowacki'
if true_author == 'Henryk Sienkiewicz':
  lime_text_path = './text_examples/sienkiewicz.txt'
if true_author == 'Adam Mickiewicz':
  lime_text_path = './text_examples/mickiewicz.txt'
if true_author == 'Juliusz Słowacki':
  lime_text_path = './text_examples/slowacki.txt'
with open(lime_text_path) as f:
  text = f.read()
print(len(text.split()))

703


In [ ]:
pos = ['ADJ', 'NOUN', 'VERB']
preprocessing_list = ['anonymize', 'lower_text', 'remove_stop_words'] # 'lower_text' / 'POS_leave_only' / 'lemmatize_text' / 'remove_punctuation' /'remove_stop_words' / 'anonymize'
text = expl.text_preprocessing(text, preprocessing_list, pos)
print(text)
print(len(text.split()))

łaskawa . 
wyznaję otwarcie, laty, otrzymaniu odpowiedzi pierwszy list, pomyślałem, żem omylił porę trafił wszelkie niepospolite dary, mogły złożyć wysoce harmonijną całość, znajdowały przynajmniej owej chwili, stanie rozstroju. odpowiedź mogła zachęcić dalszego badania, , następne podróże odjęły możność. uważałem, jednorazowej wymianie listów poprzestać prostej przyczyny, życzy, wróciwszy zwykłych obowiązków życia, starałem myśleć. upływie lat mogę absolutnie twierdzić, czym kim mówił - pozwalam wątpić, albowiem nieskłonny wynurzeń osobistych rzeczach rozmawiam. 

wyznaję , odpowiedź, jaką niegdyś otrzymałem pierwszy list, sprawiła przykrość. pierwszej chwili poznania uczułem wyjątkowo żywą sympatię, albowiem przypomniała - tyle rysami twarzy postacią głosem, drogą zmarłą osobę. powodu chciałem zbliżyć zmienić wypadkowe spotkanie bliższą znajomość. czyniłem ubocznych myśli obliczeń, zupełną prostotą chęci, ówczesnym osamotnieniu mieć choćby krańcu placename przyjazną kobiecą duszę, kt

In [ ]:
for i in range(len(representations) * len(models)):
  print(expl.classify(text, i))

In [ ]:
expl.lime(text, 0, lime_path(true_author), true_author)

In [ ]:
expl.lime(text, 1, lime_path(true_author), true_author)

In [ ]:
expl.lime(text, 2, lime_path(true_author), true_author)

Propozycja dalszego rozwoju eksperymentów - nieopisany w pracy SHAP

In [ ]:
expl.shap(text,0)

In [ ]:
expl.shap(text,2)